In [1]:
%load_ext autoreload
%autoreload 2
%load_ext Cython
%reload_ext Cython

In [3]:
import pandas as pd
import numpy as np
import pyarrow.parquet as pq
import time
import datetime as dt
from pathlib import Path
from data_functions import  create_causal_data
from multi_outcomes import multi_outcomes_data


#### Adjust pandas Outcput

In [4]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

#### LOAD DATA:
1. <b>CANARY(TREATED)
2. <b>BIOMETRICS(OUTCOME)
3. <b>PMPM(ELIGIBLE)

In [5]:
%%time
canary_p = '../../temp/CANARY.p'
biometrics_p = '../../data/BIOMETRICS'
pmpm_p = '../../temp/PMPM.p'

canary = pd.read_pickle(canary_p)
biometrics = pq.read_table(biometrics_p).to_pandas()
pmpm = pd.read_pickle(pmpm_p)


CPU times: user 26.6 s, sys: 9.22 s, total: 35.8 s
Wall time: 19 s


In [6]:
pmpm.head(2)

PERS_ID  AGE_AT_MIDMONTH  ALLOW_AMT  CC_ADHD  CC_ALZHEIMER  CC_ALZHEIMER_PLUS  CC_AMI  CC_ANEMIA  CC_ANXIETY_DISORDER  CC_ARTHRITIS  CC_ASTHMA  CC_ATRIAL_FIB  CC_AUTISM  CC_BIPOLAR  CC_BPH  CC_BREAST_CANCER  CC_CATARACT  CC_CEREBRAL_PALSY  CC_CHRONIC_KIDNEY  CC_COLORECTAL_CANCER  CC_COPD  CC_CYSTIC_FIBROSIS  CC_DEPRESSION  CC_DEPRESSIVE_DISORDERS  CC_DEVELOP_DELAYS  CC_DIABETES  CC_DISAB_DX_CNT  CC_DX_CNT  CC_ENDOMETRIAL_CANCER  CC_EPILEPSY  CC_FIBROMYALGIA  CC_GLAUCOMA  CC_HEARING_IMPAIR  CC_HEART_FAILURE  CC_HEPATITIS_A  CC_HEPATITIS_B_ACUTE  CC_HEPATITIS_B_CHRONIC  CC_HEPATITIS_C_ACUTE  CC_HEPATITIS_C_CHRONIC  CC_HEPATITIS_C_UNSPECIFIED  CC_HEPATITIS_D  CC_HEPATITIS_E  CC_HEPATITIS_GEN  CC_HIP_FRACTURE  CC_HYPERLIPIDEMIA  CC_HYPERTENSION  CC_HYPOTHYROID  CC_INT_DISAB  CC_ISCHEMIC  CC_LEARN_DISAB  CC_LEUKEMIAS  CC_LIVER  CC_LUNG_CANCER  CC_MIGRAINE  CC_MOBILITY_IMPAIR  CC_MULTIPLE_SCLEROSIS  CC_MUSCULAR_DYSTROPHY  CC_OBESITY  CC_OSTEOPOROSIS  CC_PERIPHERAL  CC_PERSONALITY  \
0  2570122               95      40392        0             0                  1       0          1                    0             0          0              1          0           0       1                 0            0                  0                  1                     0        1                   0              1                        0                  0            0                0         12                      0            0                0            1                  0                 1               0                     0                       0                     0                       0                           0               0               0                 0                0                  0                0               1             0            1               0             0         0               0            0                   0                      0                      0           0                0              0               0   
1  2570122               94      19837        0             0                  0       0          1                    0             1          0              0          0           0       1                 0            0                  0                  0                     0        1                   0              0                        0                  0            0                0          7                      0            0                0            1                  0                 0               0                     0                       0                     0                       0                           0               0               0                 0                0                  0                0               1             0            1               0             0         0               0            0                   0                      0                      0           0                0              0               0   

   CC_PRESSURE  CC_PROSTATE_CANCER  CC_PTSD  CC_SCHIZOPHRENIA  CC_SCHIZOPHRENIA_OTHER  CC_SPINAL_CORD  CC_SPINA_BIFIDA  CC_STROKE  CC_TOBACCO  CC_TRAUMATIC_BRAIN  CC_VISUAL_IMPAIR MBR_GNDR     MYR  
0            0                   0        0                 0                       1               0                0          0           0                   0                 0        M  202002  
1            0                   0        0                 0                       0               0                0          0           0                   0                 0        M  201810

#### An inspection of categories registered in biometrics data

In [7]:
%%time
pd.DataFrame(biometrics.groupby(by = 'BIOMETRIC_CATEGORY').apply(len)).T

CPU times: user 19.5 s, sys: 4.97 s, total: 24.5 s
Wall time: 24.5 s


BIOMETRIC_CATEGORY     A1C       BMI    BNP  Blood Pressure (Diastolic)  Blood Pressure (Systolic)  Cholesterol (LDL)  DCSI-EGFR  DCSI-MICROALBUMIN  DCSI-SERUM  Ferritin    Height  Hematocrit  Hemoglobin    Iron  NT_proBNP     PSA  Potassium  Troponin    Weight    eGFR
0                   790853  10034183  42552                     4902004                    5238991            1305880     628758              81297     2734084    219613  11808045     2574412     2919909  197878       9258  310283    2666981     94030  14000267  907656

## Steps in creating causal data
``` python
%%time
from data_functions import  biometrics_f, pmpm_f, canary_f, merge_filter_separate, control_outcomes, treated_outcomes, combine_treated_controls
biometrics = biometrics_f(biometrics, 'BMI', True)
pmpm = pmpm_f(pmpm, True, 12)
canary = canary_f(canary)
merged, treated, controls, canary = merge_filter_separate(biometrics, pmpm, canary, merge_how = 'inner')
controls_final = control_outcomes(controls, max_days_apart = 180, min_days_apart = 120)
treated_final, treated_help = treated_outcomes(treated, max_days_apart = 180, min_days_apart = 120)
df = combine_treated_controls(treated_final, control_finals)
```

## Multi Outcomes Data

In [8]:
%%time
kwargs_list = [{'enroll_months':12, 'drop_single':False, 'merge_how': 'left',
                'threshold': 27, 'bigger_better':False},
          {'drop_single':False, 'merge_how': 'left', 'threshold': 5.8, 'bigger_better':False}
         ]
          
#biometrics_list = ['BMI', 'A1C', 'Blood Pressure (Diastolic)', 'Blood Pressure (Systolic)', 'Cholesterol (LDL)', 'Weight']
biometrics_list = ['BMI', 'A1C']
data = multi_outcomes_data(pmpm.copy(), biometrics.copy(),
                           canary.copy(), 
                           biometrics_list = biometrics_list,
                           kwargs_list = kwargs_list)

A total of 2 iterations will be run


/mnt/home/gmavroud/rpi-canary-eval/notebooks/causal_data_creation/data_functions.py:477: RuntimeWarning: invalid value encountered in less
  outcomes = ((res_value[1:]-res_value[0:-1]) < 0).astype(int)


Iteration 0 of 2
Created the data for biometrics with name: BMI


/mnt/home/gmavroud/rpi-canary-eval/notebooks/causal_data_creation/data_functions.py:478: RuntimeWarning: divide by zero encountered in true_divide
  outcome2 = ((res_value[1:]-res_value[0:-1])/res_value[0:-1])
/mnt/home/gmavroud/rpi-canary-eval/notebooks/causal_data_creation/data_functions.py:478: RuntimeWarning: invalid value encountered in true_divide
  outcome2 = ((res_value[1:]-res_value[0:-1])/res_value[0:-1])


Iteration 1 of 2
Created the data for biometrics with name: A1C
CPU times: user 7min 44s, sys: 59.6 s, total: 8min 44s
Wall time: 8min 41s


In [9]:
def combine_outcomes(data, func = None):
    
    def orf(x):
        return (x.prod() > 0).astype(int)
    
    if func is None:
        func = orf
        
    df = pd.DataFrame(data[0].PERS_ID)
    for i, d in enumerate(data):
        df['Outcome_'+str(i)] = d['OUTCOME3'].values
    
    df['OUTCOME4'] = df.iloc[:,1:].apply(func, axis = 1)
    
    final = data[0]
    final['OUTCOME4'] = df.OUTCOME4.copy()
    return final,  df[['PERS_ID', 'OUTCOME4']]

    

In [10]:
df, _ = combine_outcomes(data.copy(), func = None)

In [11]:
Path('../../matching_data').mkdir(parents = True, exist_ok = True)
df.to_pickle('../../matching_data/data.p')


### DoWhy demonstration on the data(install dowhy with)
```pip3 install dowhy --user```  
or  
```!pip3 install dowhy --user```  directly through the notebook

!pip3 install dowhy --user

In [12]:
import dowhy
from dowhy import CausalModel

import logging
logging.getLogger("dowhy").setLevel(logging.WARNING)
import warnings
from sklearn.exceptions import DataConversionWarning
warnings.filterwarnings(action='ignore', category=DataConversionWarning)

### Specify Common Causes Names

In [13]:
common_causes = np.setdiff1d(df.columns, ['BIOMETRIC_CATEGORY', 'OUTCOME', 'OUTCOME2', 'OUTCOME3', 'OUTCOME4'
                                          'PERS_ID', 'RESULT_VALUE', 'TREATED', 'RESULT_VALUE2'])

df.TREATED = df.TREATED.astype(bool)


### Create data dictionary for causal modeling

In [14]:
data1 = {'df':df, 
        'common_causes_names':list(common_causes), 
        'outcome1':'OUTCOME', 
        'outcome2':'OUTCOME1',
        'outcome3':'OUTCOME3',
         'outcome4':'OUTCOME4',
        'treatment_name':'TREATED'}



In [15]:
def causal_analysis(data, outcome_name,  method = "backdoor.propensity_score_stratification"):
    
    #define causal model
    model = CausalModel(
            data=data['df'],
            treatment=data['treatment_name'],
            outcome=data[outcome_name],
            common_causes=data['common_causes_names'],
            )
    
    #check if it is possible the estimand to be estimated
    identified_estimand = model.identify_effect(proceed_when_unidentifiable=True)
    
    #calculate propensities
    propensity_match_estimate = model.estimate_effect(identified_estimand,
                            method_name = method,
                            test_significance = False,
                            confidence_intervals = False,
                            target_units="att")
    
    return propensity_match_estimate



In [16]:
m1 = causal_analysis(data1, 'outcome4')


### JOINT RESULTS

In [17]:
m1.value